# Exploratory Analysis
---
In this notebook, we explore the dataset collected from Amazon.

We load the required packages, including functions defined for feature extraction of the book covers:

In [1]:
from matplotlib import pyplot as plt
from feature_extraction import *
from functools import reduce

import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import math

import colorgram
import cv2
import csv
import os
import re

%matplotlib inline

Data and book covers are saved in:

In [2]:
collected_data_folder = './Collected Data/'
original_data_folder = './Original Data/'
image_folder = './Collected Data/Cover Images/'

### Overview

We have chosen to divide the project into the following subtasks:
1. Extract semantically meaningful features of the book covers,
2. Train a model with a good fit of the response variable,
3. Extract information from the model regarding most important features.

Since it is not the aim of the project to use an algorithm that simultanously learns the features and the model at the same time (like Krizhevsky et al.’s ImageNet CNN), we have chosen to first extract the features, and then build a model. This way we also benefit from having interpretable features.

### Extracting features

We are interested in extracting overall features through white-box algorithms for the images. Overall, the following featurs will be extracted:

- Top 2 colors in the image given as one metric storing the information of the RGB value
- Number of unique colors
- Brightness value 
- Entropy value (degree of randomness in the picture)
- Complexity value defined as number of critical points normalized by resolution

**Reading the image**

Get paths of all book covers:

In [3]:
# List of paths of book covers
train_paths = []
for root, dirs, files in os.walk(image_folder):
    for file in files:
        # Ignore hidden files
        if not file.startswith('.'):
            train_paths.append((os.path.join(root, file)))

To be able to stop and continue at will, we will write the computed features to a csv file as we go along. Let's initialize this file with a meaningful header:

In [4]:
with open('features.csv', 'a') as file:
    writer = csv.writer(file)
    writer.writerow(['BookID', 'Brightness', 'Colorfulness', 'Entropy', 'UniqueColors', 'Keypoints',
                     'Color1R', 'Color1G', 'Color1B', 'Color2R', 'Color2G', 'Color2B'])

Load book covers and extract features using pre-defined functions:

In [74]:
# Allocate feature matrix
num_colors = 2
num_features = 5 + num_colors*3
#X = np.zeros((len(train_paths), num_features)) 
IDs = [path[30:40] for path in train_paths]

with open('features.csv', 'a+') as file:
    reader = csv.reader(file)
    writer = csv.writer(file)
    
    # Look at the last scraped book to continue from the next one in the dataset
    file.seek(0)
    last_scraped = next(reversed(list(reader)))[0]
    
    if(last_scraped == 'BookID'):
        # Nothing was scraped yet, start from the beginning
        index = 0
    else:
        # At least one book was scraped, find the index of the last scraped book and start from the next one
        last_scraped_index = IDs.index(last_scraped)
        index = last_scraped_index + 1
        
    count = 0  
    skip_count = 0
    while(index < len(IDs)):
        current_id = IDs[index]

        path = image_folder + current_id + '.jpg'
        image = cv2.imread(path)
        
        # Make sure that unwanted image files does not corrupt
        if image is None:
            index += 1
            skip_count += 1
            continue
        
        X = []
        
        # Book ID
        X.append(path[30:40])
        
        # Exract brightness of image
        X.append(calculate_brightness(image))

        # Extract colorfulness metric of image
        X.append(calculate_colorfulness(image))

        # Calculate image entropy as a measure of degree of randomness
        X.append(calculate_entropy(image))

        # Compute number of unique colors in image
        X.append(calculate_num_unique_colors(image))

        # Calculate number of keypoints in image
        X.append(calculate_num_norm_keypoints(image))

        # Extract RGB color codes of top 2 dominant colors of image
        X = np.r_[X, calculate_top_colors(path, num_colors)]
        
        # Write to file
        writer.writerow(X)
            
        file.flush()

        index += 1
        count += 1    
            
        # Clean the previous line while printing info about scraping progress
        print(f'Number of scraped books: {count}    Number of skipped books: {skip_count}', end='\r')
        

We create a dataframe and inspect the top rows:

In [11]:
covers = pd.read_csv('features.csv', sep=',')
covers.rename(columns = {'BookID':'ID'}, inplace = True)
covers.head()

ID  Brightness  Colorfulness   Entropy  UniqueColors  Keypoints  \
0  0689871872    0.393601     30.900888  7.208404         55245   0.025003   
1  0802774067    0.631390     90.699536  7.118225        123764   0.014360   
2  1442496797    0.525140    122.037335  6.843093         56732   0.018625   
3  0375824308    0.508363     62.399971  7.407145         95994   0.024494   
4  0395899257    0.841409    112.527855  3.904536         43675   0.007876   

   Color1R  Color1G  Color1B  Color2R  Color2G  Color2B  
0       35       42       68      117       91       75  
1      188      236      193      208      240      166  
2        4      185      197        3       90      108  
3      180      163      120       41      135      101  
4      252      252       76      252      252      244

#### Merging the datasets

We now merge the book cover features with the remaining two datasets. First, we load them:

In [12]:
# Load scraped data
scraped = pd.read_csv(collected_data_folder + 'scraped.csv', sep=',')

# Load original book dataset with header names
header_names = ['ID', 'Filename', 'Image URL', 'Title', 'Author', 'Category ID', 'Category']
original = pd.read_csv(original_data_folder + 'book32-listing.csv', encoding='latin1', header=None, names=header_names)

For some reason, the ID column in the original dataset has the leading 0s removed (normally all of them should be 10 characters long). The filename column has the correct IDs with the correct number of leading 0s. So let's use the Filename column as the new ID column:

In [13]:
original['ID'] = original['Filename'].apply(lambda row: re.findall(u'(.*).jpg', row)[0])
original.drop(columns='Filename', inplace=True)
original.head()

ID                                          Image URL  \
0  0761183272  http://ecx.images-amazon.com/images/I/61Y5cOdH...   
1  1623439671  http://ecx.images-amazon.com/images/I/61t-hrSw...   
2  B00O80WC6I  http://ecx.images-amazon.com/images/I/41X-KQqs...   
3  0761182187  http://ecx.images-amazon.com/images/I/61j-4gxJ...   
4  1578052084  http://ecx.images-amazon.com/images/I/51Ry4Tsq...   

                                               Title              Author  \
0                    Mom's Family Wall Calendar 2016      Sandra Boynton   
1                    Doug the Pug 2016 Wall Calendar        Doug the Pug   
2  Moleskine 2016 Weekly Notebook, 12M, Large, Bl...           Moleskine   
3            365 Cats Color Page-A-Day Calendar 2016  Workman Publishing   
4               Sierra Club Engagement Calendar 2016         Sierra Club   

   Category ID   Category  
0            3  Calendars  
1            3  Calendars  
2            3  Calendars  
3            3  Calendars  
4            3  Calendars

Now we inspect the top rows to make sure that nothing is corrupted:

In [14]:
scraped.head()

ID Sales Rank               Date Review Score
0  0545790352        118    October 6, 2015         4.77
1  1419717014        399   November 3, 2015          4.8
2  1423160916       9637    October 6, 2015          4.6
3  1476789886       5439   October 27, 2015          4.9
4  1338029991        196  November 10, 2015         4.61

In [15]:
original.head()

ID                                          Image URL  \
0  0761183272  http://ecx.images-amazon.com/images/I/61Y5cOdH...   
1  1623439671  http://ecx.images-amazon.com/images/I/61t-hrSw...   
2  B00O80WC6I  http://ecx.images-amazon.com/images/I/41X-KQqs...   
3  0761182187  http://ecx.images-amazon.com/images/I/61j-4gxJ...   
4  1578052084  http://ecx.images-amazon.com/images/I/51Ry4Tsq...   

                                               Title              Author  \
0                    Mom's Family Wall Calendar 2016      Sandra Boynton   
1                    Doug the Pug 2016 Wall Calendar        Doug the Pug   
2  Moleskine 2016 Weekly Notebook, 12M, Large, Bl...           Moleskine   
3            365 Cats Color Page-A-Day Calendar 2016  Workman Publishing   
4               Sierra Club Engagement Calendar 2016         Sierra Club   

   Category ID   Category  
0            3  Calendars  
1            3  Calendars  
2            3  Calendars  
3            3  Calendars  
4            3  Calendars

It looks fine for now, so we continue to merge the all the datasets using the book IDs as our primary key: 

In [231]:
dfs = [covers, original, scraped]
books = reduce(lambda left,right: pd.merge(left,right,on='ID'), dfs)
books.head()

ID  Brightness  Colorfulness   Entropy  UniqueColors  Keypoints  \
0  0689871872    0.393601     30.900888  7.208404         55245   0.025003   
1  0802774067    0.631390     90.699536  7.118225        123764   0.014360   
2  1442496797    0.525140    122.037335  6.843093         56732   0.018625   
3  0375824308    0.508363     62.399971  7.407145         95994   0.024494   
4  0395899257    0.841409    112.527855  3.904536         43675   0.007876   

   Color1R  Color1G  Color1B  Color2R  Color2G  Color2B  \
0       35       42       68      117       91       75   
1      188      236      193      208      240      166   
2        4      185      197        3       90      108   
3      180      163      120       41      135      101   
4      252      252       76      252      252      244   

                                           Image URL  \
0  http://ecx.images-amazon.com/images/I/51kbkQ35...   
1  http://ecx.images-amazon.com/images/I/51HWF41Q...   
2  http://ecx.images-amazon.com/images/I/51M%2BHR...   
3  http://ecx.images-amazon.com/images/I/51D%2BOM...   
4  http://ecx.images-amazon.com/images/I/51frwULt...   

                                               Title            Author  \
0  Althea Gibson: Young Tennis Player (Childhood ...  Beatrice Gormley   
1  This Place Is High: The Andes Mountains of Sou...        Vicki Cobb   
2  The Case of the Weird Blue Chicken: The Next M...     Doreen Cronin   
3  Sergei Prokofiev's Peter and the Wolf: With a ...  Sergei Prokofiev   
4                              Curious George's ABCs         H. A. Rey   

   Category ID          Category Sales Rank                Date Review Score  
0            4  Children's Books    2642216     January 6, 2005            0  
1            4  Children's Books    2230093   September 1, 1993          4.0  
2            4  Children's Books     878409  September 30, 2014         4.78  
3            4  Children's Books        NaN                 NaN          NaN  
4            4  Children's Books     639464      March 30, 1998         4.59

We only have Children's Books so we don't need the Category or Category ID columns anymore:

In [232]:
books.drop(columns=['Category ID', 'Category'], inplace=True)
books.head()

ID  Brightness  Colorfulness   Entropy  UniqueColors  Keypoints  \
0  0689871872    0.393601     30.900888  7.208404         55245   0.025003   
1  0802774067    0.631390     90.699536  7.118225        123764   0.014360   
2  1442496797    0.525140    122.037335  6.843093         56732   0.018625   
3  0375824308    0.508363     62.399971  7.407145         95994   0.024494   
4  0395899257    0.841409    112.527855  3.904536         43675   0.007876   

   Color1R  Color1G  Color1B  Color2R  Color2G  Color2B  \
0       35       42       68      117       91       75   
1      188      236      193      208      240      166   
2        4      185      197        3       90      108   
3      180      163      120       41      135      101   
4      252      252       76      252      252      244   

                                           Image URL  \
0  http://ecx.images-amazon.com/images/I/51kbkQ35...   
1  http://ecx.images-amazon.com/images/I/51HWF41Q...   
2  http://ecx.images-amazon.com/images/I/51M%2BHR...   
3  http://ecx.images-amazon.com/images/I/51D%2BOM...   
4  http://ecx.images-amazon.com/images/I/51frwULt...   

                                               Title            Author  \
0  Althea Gibson: Young Tennis Player (Childhood ...  Beatrice Gormley   
1  This Place Is High: The Andes Mountains of Sou...        Vicki Cobb   
2  The Case of the Weird Blue Chicken: The Next M...     Doreen Cronin   
3  Sergei Prokofiev's Peter and the Wolf: With a ...  Sergei Prokofiev   
4                              Curious George's ABCs         H. A. Rey   

  Sales Rank                Date Review Score  
0    2642216     January 6, 2005            0  
1    2230093   September 1, 1993          4.0  
2     878409  September 30, 2014         4.78  
3        NaN                 NaN          NaN  
4     639464      March 30, 1998         4.59

In [233]:
books.shape

(13478, 18)

Ta-da! It looks fine, so we proceed with the exploratory analysis.

#### Exploring the dataset

We see that we have a few missing values. Let's check how many we have:

In [234]:
books.isnull().sum()[books.isnull().sum() > 0]

Author           436
Sales Rank      1051
Date             989
Review Score     943
dtype: int64

We check if the missing values occur in the same rows: 

In [235]:
missing = books[books['Author'].isna() & books['Sales Rank'].isna() & books['Date'].isna() & books['Review Score'].isna()]
missing

ID  Brightness  Colorfulness   Entropy  UniqueColors  \
341    0439079462    0.479689     70.433117  7.478615         85699   
893    1908985313    0.627805    141.541011  5.748841         56554   
919    0736421394    0.700083     53.831143  6.540955         54460   
929    0970863349    0.435797     82.148774  7.382773         67646   
1104   0887432719    0.685996     79.946071  6.485711         66804   
2643   0486462005    0.595610     50.351391  7.809225        107279   
2951   1846667747    0.671400    102.872033  7.367915        101459   
2980   1481447998    0.392777     58.733583  7.742380         97965   
3345   0590100548    0.361937     68.937413  5.835538         42636   
3538   1481462202    0.683949     90.405098  7.211342        100653   
4807   043977375X    0.595875     53.411090  7.365581         64035   
4931   1771470038    0.553832     74.188857  7.004644         52909   
4979   1423139836    0.414107     51.625038  7.302373         54352   
6161   1438073429    0.632194     88.736887  7.034083         59488   
6211   0486452204    0.497252     84.057163  7.588954         95455   
6544   1438073399    0.581593    112.703184  6.739918         67418   
7166   0843198974    0.361763     70.312403  6.728172         40906   
7633   0486447014    0.662377     85.380647  6.372495         64515   
7667   1592701655    0.710423     48.307287  6.728006         17331   
7908   0399216197    0.304252    119.234172  6.731634         43808   
8051   1846660939    0.667647     92.219177  7.463008         98651   
8663   0439773768    0.740209     47.943523  7.272566         78923   
9516   0307960307    0.749035     90.501203  7.176841         73995   
9859   1481448196    0.324860     62.195760  7.664389         90228   
10589  048645178X    0.697210     47.146155  7.551963         73505   
10638  0764164260    0.707796     13.347721  6.814631         16256   
10888  1499800401    0.684940     50.939796  6.010003         59156   
12637  1481438565    0.606477     94.535598  7.656355         92448   
13185  0761158626    0.534465     92.666745  7.117960        119871   
13427  1481441728    0.588803     80.858709  7.683881        112366   

       Keypoints  Color1R  Color1G  Color1B  Color2R  Color2G  Color2B  \
341     0.018883       53       97       84       32       52       45   
893     0.008629      252      202       27      250      172       29   
919     0.018157      232      233      210      128      169      134   
929     0.012267      179      150      252      149      126      246   
1104    0.015243      253      254      250      158      194       67   
2643    0.065553      238      238      209      117       95       65   
2951    0.031679      123      200      185      248      242      223   
2980    0.028725       54      117       58       24       25       38   
3345    0.008316      124       54       72       65       82      159   
3538    0.030048      251      246      237      209      165       82   
4807    0.021971      213      203      215      104       80       69   
4931    0.031627      215      170      123       24       16       11   
4979    0.037193      132       90       59       61       47       31   
6161    0.003005      224      226       81       64      133      198   
6211    0.025179      157       84       20      146      157       18   
6544    0.004639      244      234       75       38      105      173   
7166    0.013073      123       82       60       17       12       10   
7633    0.019362      144      194      147      240      211       61   
7667    0.007783      190      220      240       46      109      141   
7908    0.007738      243       73       39       43       27       22   
8051    0.025573      189      165      123      156       98       40   
8663    0.022856      211      232      246      172      175      144   
9516    0.018638      226      233      246      240      211       74   
9859    0.027679       22       25   

In [55]:
len(missing)

943

As a quick fix, we decide to drop the missing values and continue with the exploratory analysis.

In [236]:
books = books.dropna().reset_index()
len(books)

12017

Now we check the different types:

In [237]:
books.dtypes

index             int64
ID               object
Brightness      float64
Colorfulness    float64
Entropy         float64
UniqueColors      int64
Keypoints       float64
Color1R           int64
Color1G           int64
Color1B           int64
Color2R           int64
Color2G           int64
Color2B           int64
Image URL        object
Title            object
Author           object
Sales Rank       object
Date             object
Review Score     object
dtype: object

Let's convert the RGB columns from floats to integers: 

In [238]:
# Column names
rgb_features = ['Color1R', 'Color1G', 'Color1B', 'Color2R', 'Color2G', 'Color2B']

# Convert to integers
for rgb_feature in rgb_features:
    books[rgb_feature] = books[rgb_feature].astype(int)

We inspect the dates and check if they are easily convertible to a datetime format. We know, that the date variable is of the form `Month Date, Year`, which means the date entries of this format will have a known lengths. Let's compute the possible lengths:

In [239]:
months = ['January', 'February', 'March', 'April', 'May', 
          'June', 'July', 'August', 'September', 'October', 'November', 'December']
char = len(',') + len('year') + len('  ')
start_month_dates = [len(month) + 1 + char for month in months]
end_month_dates = [start_month_dates[i] + 1 for i in range(len(start_month_dates))]
lengths = list(set(start_month_dates + end_month_dates))
lengths

[11, 12, 13, 14, 15, 16, 17, 18]

We print the rows that do not follow this format:

In [240]:
for i in range(len(books['Date'])):
    if len(books['Date'][i]) not in lengths:
        print(i,': ',books['Date'][i])

101 :  1992
250 :  2005
331 :  May 1999
352 :  March 2003
412 :  1996
532 :  2002
614 :  2012
734 :  1978
946 :  June 2005
1031 :  1995
1033 :  1999
1398 :  March 2000
1421 :  1920
1475 :  2011
1613 :  March 2004
1720 :  April 2002
1781 :  1989
1920 :  March 2004
1997 :  March 2002
2193 :  1989
2321 :  2001
2342 :  April 2003
2371 :  2009
2448 :  June 2009
2476 :  1986
2608 :  2012
2625 :  2007
2676 :  1995
2739 :  March 2002
2779 :  1984
2864 :  2007
3058 :  July 2009
3137 :  April 2005
3184 :  April 1998
3197 :  2006
3354 :  March 2003
3474 :  2002
3651 :  March 1994
3891 :  March 2002
4081 :  June 2005
4296 :  April 2002
4382 :  1983
4479 :  2015
4480 :  2015
4527 :  April 2002
4638 :  2009
4705 :  March 2001
4930 :  1991
5090 :  2014
5109 :  March 2003
5261 :  April 2002
5302 :  2007
5366 :  March 2003
5378 :  2002
5405 :  July 2001
5547 :  April 2001
5607 :  June 2005
5851 :  2010
6017 :  March 2003
6063 :  2007
6254 :  April 2005
6366 :  2011
6638 :  2002
6696 :  2002
6888 :  199

In [241]:
#books['Date'] = pd.to_datetime(books['Date'])

Also, the sales rank and the review score are stored as categorical features. We check what non-numerical entries we have:

In [242]:
non_num = books[~books['Sales Rank'].str.match('^\-?(\d*\.?\d+|\d+\.?\d*)$') | ~books['Review Score'].str.match('^\-?(\d*\.?\d+|\d+\.?\d*)$')]
non_num

index          ID  Brightness  Colorfulness  Entropy  UniqueColors  \
9916  11106  1505378087    0.292872     83.422735  5.60072         45881   

      Keypoints  Color1R  Color1G  Color1B  Color2R  Color2G  Color2B  \
9916   0.015219        5      128        4        4       57        5   

                                              Image URL  \
9916  http://ecx.images-amazon.com/images/I/51DzGaKM...   

                                 Title                   Author Sales Rank  \
9916  The Ultimate Players Guide To MC  Ultimate App Guidebooks  Error 404   

           Date Review Score  
9916  Error 404    Error 404

Only one entry! We see no other option but dropping this row: 

In [250]:
books = books.drop(9916).reset_index(inplace=True)

Evanesco!

In [251]:
books[~books['Sales Rank'].str.match('^\-?(\d*\.?\d+|\d+\.?\d*)$') | ~books['Review Score'].str.match('^\-?(\d*\.?\d+|\d+\.?\d*)$')]

Empty DataFrame
Columns: [level_0, index, ID, Brightness, Colorfulness, Entropy, UniqueColors, Keypoints, Color1R, Color1G, Color1B, Color2R, Color2G, Color2B, Image URL, Title, Author, Sales Rank, Date, Review Score]
Index: []

The sales rank and review score columns are stored as objects. Let's convert them to numerical ones:

In [254]:
books['Sales Rank'] = books['Sales Rank'].astype(float)
books['Review Score'] = books['Review Score'].astype(float)

Now, let's inspect summary statistics for the numerical features:

In [255]:
books.describe()

level_0         index    Brightness  Colorfulness       Entropy  \
count  12016.000000  12016.000000  12016.000000  12016.000000  12016.000000   
mean    6007.674767   6730.797853      0.560428     68.345483      6.784220   
std     3469.114576   3892.387035      0.152539     26.840568      0.949329   
min        0.000000      0.000000      0.040673      0.000000      1.031240   
25%     3003.750000   3356.500000      0.452572     48.974511      6.420341   
50%     6007.500000   6734.000000      0.562823     65.375971      7.078930   
75%     9011.250000  10103.250000      0.672726     84.861714      7.448454   
max    12016.000000  13477.000000      0.981620    183.419653      7.968290   

        UniqueColors     Keypoints       Color1R       Color1G       Color1B  \
count   12016.000000  12016.000000  12016.000000  12016.000000  12016.000000   
mean    69604.763316      0.024651    157.609854    153.350866    138.928096   
std     28214.416329      0.011781     83.916005     78.953381     81.271605   
min       219.000000      0.000461      0.000000      0.000000      0.000000   
25%     50098.750000      0.015784     80.750000     88.000000     63.000000   
50%     68742.500000      0.023152    180.000000    164.000000    137.000000   
75%     88600.500000      0.031646    240.000000    230.000000    219.000000   
max    184422.000000      0.084987    254.000000    254.000000    254.000000   

            Color2R       Color2G       Color2B    Sales Rank  Review Score  
count  12016.000000  12016.000000  12016.000000  1.201600e+04  12016.000000  
mean     149.661951    139.176764    120.653379  1.538024e+06      3.994402  
std       77.318945     72.782293     74.822149  2.895143e+06      1.547258  
min        0.000000      0.000000      0.000000  1.300000e+01      0.000000  
25%       83.000000     85.000000     58.000000  5.319400e+04      4.220000  
50%      159.000000    151.000000    110.000000  3.466460e+05      4.610000  
75%      221.000000    209.000000    191.000000  1.501567e+06      4.790000  
max      254.000000    254.000000    254.000000  2.147187e+07      5.000000

..and also for the categorical ones:

In [256]:
books.describe(include=[np.object])

ID                                          Image URL  \
count        12016                                              12016   
unique       12016                                              12005   
top     1433981815  http://ecx.images-amazon.com/images/I/51RDK0PW...   
freq             1                                                  2   

                         Title         Author               Date  
count                    12016          12016              12016  
unique                   11894           5724               3191  
top     Jack and the Beanstalk  DK Publishing  September 1, 2015  
freq                         4             86                 79

We start with examining the distribution of the response variable `Sales Rank` through a boxplot:

In [257]:
books.boxplot(column='Sales Rank');

It seems that we have few books that are very popular, and a lot of books that are not. Could this be a power law?

In [265]:
num_bins = 30

y, bin_edges = np.histogram(books['Sales Rank'], num_bins)
bin_centers = 0.5*(bin_edges[1:]+bin_edges[:-1])

plt.hist(books['Sales Rank'], num_bins);
plt.plot(bin_centers, y, lw=3);
#plt.xlabel('x');
#plt.ylabel('Sales Rank');

Let's inspect a log-log histogram:

In [266]:
plt.loglog(bin_centers, y);
#plt.xlabel('x');
#plt.ylabel('Sales Rank');

It roughly looks like one, with the linear part at the beginning and the noise at the end! Let's also look at the cumulative distribution to confirm.

In [267]:
sales = np.sort(books['Sales Rank'].values)[::-1]
ranks = range(len(sales))
plt.loglog(sales, ranks);
#plt.xlabel('x');
#plt.ylabel('#tags that appear in over x questions');

We are not completely convinced. We also examine the relationship between `Review Score` and `Sales Rank`:

In [268]:
books.plot.scatter(x='Review Score',y='Sales Rank');

A correlation between `Review Score` and `Sales Rank` is hard to detect based only on the scatter plot above. We also compute the Pearson and the Spearman correlation along with 95% confidence intervals to confirm/verify this:

In [262]:
# Variables and significance level
x = books['Review Score']
y = books['Sales Rank']
alpha = 0.05
methods = ['pearson', 'spearman'] # correlation methods 
cols = ['lower', 'correlation', 'upper']

# Get correlation coefficients
r = sp.stats.pearsonr(x, y)[0], sp.stats.spearmanr(x, y)[0]

# Transform correlation coefficients into a Fishers' Z-score
r_z = np.arctanh(r)

# Compute standard deviations
se = 1/np.sqrt(len(x) - 3)

# Compute CIs
z = sp.stats.norm.ppf(1 - alpha/2)
lower_z, upper_z = r_z - z*se, r_z + z*se
lower, upper = np.tanh((lower_z, upper_z))

# Return correlation coefficents and CIs
pd.DataFrame(np.transpose(np.array([lower, r, upper])),
             columns=[cols],
             index=[methods])

lower correlation     upper
pearson  -0.65339   -0.643022 -0.632413
spearman -0.32468   -0.308591 -0.292324

The Spearman correlation coefficient seems to indicate a negative linear relationship between the sales rank and the review score, however, the scatter plot does not appear to support this. 

### Discussion of methods

Having done the exploratory analysis, we proceed with building a model based on the above extracted semantically meaningful visual features. The response of the model will be the `Sales Rank` (and perhaps a separate model for `Review Score`).

The model being built is a Random Forest, which is an ensemble of many decision trees. The Random Forest is chosen due to a great ability of handling large datasets, reducing the risk of overfitting due to averaging the result of many trees, and for the same reason reducing the variance of the final classifier. 

The Random Forest algorithm is an _ensemble method_ that constructs a classifier by combining several independent classifiers coming from bootstrap sampling. One single decision tree has low bias but high variance—_low bias_ due to not making any assumptions about the target function before modelling, and therefore it is a very flexible model. On the other hand, the output might change as the training data changes giving it a _high variance_. 
Since each decision tree generated through bagging already will be identically distributed, the expectation of the average of these trees will be the same as the expectation of any of them, leaving the bias unchanged.

The variance will on the other hand be minimised. If variables are identically distributed (i.i.d) with variance $\sigma^2$ and the number of trees being bagged is B, the variance of the average will be $\frac{1}{B} \sigma^2$. Furthermore, if these variables are identically distributed with a positive pairwise correlation $\rho$, the variance of the average is defined to be:

$$\rho \sigma^2 + \frac{1-\rho}{B}\sigma^2$$

When B increases, the limitation of the averaging will be the size of correlation between trees. The reduction of the correlation between the trees is done through the random selection of input variables.

This bias-variance tradeoff is one of the main advantages of Random Forest. The Random Forest predictor is defined as:

$$ \hat{f}(x) = \frac{1}{B} \sum_{B=1}^B T(x,\phi_b)$$

With $T$ being the $b$'th tree that was build, and $\phi_b$ being the $b$'th tree in terms of split variables, cutpoints, etc. Since this will be used for regression, the Random Forest model works as averaging the predicted value from each tree at a given target point $x$ (The elements of Statistical Learning [chp.15] by Hastie, Tibshirani, Friedman).

As a bonus, ensembles of trees have the advantage of being quickly analyzable by [SHAP](https://github.com/slundberg/shap), which we will use to analyze our model.

### Pipeline

The future work will include:
1. Finish scraping full data set
2. Creating visual features for full data set of scraped data
3. Build a well-fitting Random Forest model
4. Examine the significance of the features and draw conclusions on book covers
    - Use the Shapley Feature Importance to represent a feature's responsibility for a change in the model output as explained in https://github.com/slundberg/shap
5. Turn these conclusions into a data story
6. Prepare a poster and presentation to convey our findings